In [15]:
#import agent
import keras
import random
import chess
import numpy as np

# wrapper class of chessbot
class Engine():
  def __init__(self, difficulty=1, model_path=''):
    
    # random
    if (difficulty == 0):
      self.is_random = True
    
      print("initialized")
      
    else:
      
      # bot difficulty
      if (difficulty == 1):
        start_model_path = model_path + '/start_net'
        end_model_path = model_path + '/end_net'
      else:
        start_model_path = model_path + '/start_net_bigset'
        end_model_path = model_path + '/end_net_bigset'
        
      self.is_random = False
      self.start_model = keras.models.load_model(start_model_path)
      self.end_model = keras.models.load_model(end_model_path)
    
      print("initialized")
    
  def get_net_input(self, fen):
        
    input = []
    last = False
    
    for c in fen:
        
        # B or W to move
        if last:
            
            if c == 'w':
                input.append(0)
            else:
                input.append(1)
            break
        
        # ignore
        if (c == '/'):
            continue
        
        # just before to move information
        elif (c == ' '):
            last = True
            continue
        
        # data about square
        else:
            
            vec = np.zeros(13) # empty square, K, Q, B, N, R, P, k, q, b, n, r, p
            
            # empty square
            try:
                
                x = int(c)
                
                vec[0] = 1
                for i in range(x):
                    for n in vec:
                        input.append(int(n))
                    
            # piece
            except:
                
                if (c == 'K'):
                    vec[1] = 1
                elif (c == 'Q'):
                    vec[2] = 1
                elif (c == 'B'):
                    vec[3] = 1
                elif (c == 'N'):
                    vec[4] = 1
                elif (c == 'R'):
                    vec[5] = 1
                elif (c == 'P'):
                    vec[6] = 1
                elif (c == 'k'):
                    vec[7] = 1
                elif (c == 'q'):
                    vec[8] = 1
                elif (c == 'b'):
                    vec[9] = 1
                elif (c == 'n'):
                    vec[10] = 1
                elif (c == 'r'):
                    vec[11] = 1
                elif (c == 'p'):
                    vec[12] = 1
                
                for n in vec:
                    input.append(int(n))
    
    return input
    
  def get_random_move(self, board):
    legal = [m for m in board.legal_moves]   
    engine_move = random.choice(legal)
    print("Engine move " + str(engine_move))
    board.san_and_push(engine_move)
    return engine_move
    
  def get_move(self, board):
    print('get move')
    if (self.is_random):
      return self.get_random_move(board)
    
    wrong_start = False
        
    # get starting square
    
    fen = board.fen()
    x_start = self.get_net_input(fen)
    
    y_start = self.start_model.predict([x_start])
    start_square_list = y_start[0]
    
    for i in range(64):
        x_start.append(0)
    
    while True:
    
        start_square_index = np.argmax(start_square_list)
        start_square = chess.SQUARE_NAMES[start_square_index]
        
        # modify x
        
        x_start[833 + start_square_index] = 1
        
        # get end square
        
        y_end = self.end_model.predict([x_start])
        end_square_list = y_end[0]
        
        while True:
        
            if wrong_start:
                break
        
            end_square_index = np.argmax(end_square_list)
            end_square = chess.SQUARE_NAMES[end_square_index]
            
            move = start_square + end_square
            
            try:
                board.push_uci(move)
                return move
            except:
                
                try:
                    board.push_uci(move + "Q")
                    return move + "Q"
                except:
                    pass
                
                if end_square_list[end_square_index] <= 0:
                    wrong_start = True
                else: 
                    end_square_list[end_square_index] = 0
                
                continue
        
        start_square_list[start_square_index] = 0
        x_start[833 + start_square_index] = 0
        wrong_start = False
    

In [16]:
import chess
from IPython.display import clear_output, display
from ipywidgets import widgets

board=chess.Board()
disp = display(display_id=True)
disp.update(board)

bot = Engine(0, '.')

text = widgets.Text(
    value='e4',
    placeholder='move',
    description='Move:',
    disabled=False
)
display(text)

def callback(wdgt):
    print("Human move " + wdgt.value)
    board.push_san(wdgt.value) # this is human move
    disp.update(board)
    move = bot.get_move(board)
    #print('bot move ' + move)
    # board.push_san(move)
    disp.update(board)

text.on_submit(callback)

initialized


Text(value='e4', description='Move:', placeholder='move')

C:\Users\sam67\AppData\Local\Temp\ipykernel_10156\3578698519.py:28: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  text.on_submit(callback)
